# 第1章: グラフの基本要素

## 準備

以下のセルを順番に実行して、演習に必要な環境をセットアップします。

### LLMプロバイダーの選択

このセルでは、使用するLLMプロバイダーを選択します。
`LLM_PROVIDER` 変数に、利用したいプロバイダー名を設定してください。
選択可能なプロバイダー: `"openai"`, `"azure"`, `"google"` (Vertex AI), `"google_genai"` (Gemini API), `"anthropic"`, `"bedrock"`

In [ ]:
# === LLMプロバイダーの選択 ===
# 利用したいLLMプロバイダーを以下の変数で指定してください。
# "openai", "azure", "google" (Vertex AI), "google_genai" (Gemini API), "anthropic", "bedrock" のいずれかを選択できます。
LLM_PROVIDER = "openai"  # 例: OpenAI を利用する場合

### APIキー/環境変数の設定

以下のセルを実行する前に、選択したLLMプロバイダーに応じたAPIキーまたは環境変数を設定する必要があります。

**手順:**
1.  `.env.sample` ファイルをコピーして `.env` ファイルを作成します。
2.  `.env` ファイルを開き、選択したLLMプロバイダーに対応するAPIキーや必要な情報を記述します。
    *   **OpenAI:** `OPENAI_API_KEY`
    *   **Azure OpenAI:** `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT`, `OPENAI_API_VERSION`, `AZURE_OPENAI_DEPLOYMENT_NAME`
    *   **Google (Vertex AI):** `GOOGLE_CLOUD_PROJECT_ID`, `GOOGLE_CLOUD_LOCATION` (Colab環境外で実行する場合、`GOOGLE_APPLICATION_CREDENTIALS` 環境変数の設定も必要になることがあります)
    *   **Google (Gemini API):** `GOOGLE_API_KEY`
    *   **Anthropic:** `ANTHROPIC_API_KEY`
    *   **AWS Bedrock:** `AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_REGION_NAME` (IAMロールを使用する場合は、これらのキー設定は不要な場合がありますが、リージョン名は必須です)
3.  ファイルを保存します。

**Google Colab を使用している場合:**
上記の `.env` ファイルを使用する代わりに、Colabのシークレットマネージャーに必要なキーを登録してください。
例えば、OpenAIを使用する場合は `OPENAI_API_KEY` という名前でシークレットを登録します。
Vertex AI を利用する場合は、Colab上での認証 (`google.colab.auth.authenticate_user()`) が実行されます。

このセルは、設定された情報に基づいて環境変数をロードし、LLMクライアントを初期化します。

In [ ]:
# === APIキー/環境変数の設定 ===
import os
from dotenv import load_dotenv

# .envファイルから環境変数を読み込む (存在する場合)
load_dotenv()

try:
    from google.colab import userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

# --- OpenAI ---
if LLM_PROVIDER == "openai":
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY and IS_COLAB:
        OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY:
        raise ValueError("OpenAI APIキーが設定されていません。環境変数 OPENAI_API_KEY を設定するか、Colab環境の場合はシークレットに OPENAI_API_KEY を設定してください。")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# --- Azure OpenAI ---
elif LLM_PROVIDER == "azure":
    AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
    AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
    OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
    AZURE_OPENAI_DEPLOYMENT_NAME = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if IS_COLAB:
        if not AZURE_OPENAI_API_KEY: AZURE_OPENAI_API_KEY = userdata.get("AZURE_OPENAI_API_KEY")
        if not AZURE_OPENAI_ENDPOINT: AZURE_OPENAI_ENDPOINT = userdata.get("AZURE_OPENAI_ENDPOINT")
        if not OPENAI_API_VERSION: OPENAI_API_VERSION = userdata.get("OPENAI_API_VERSION") # 例: "2023-07-01-preview"
        if not AZURE_OPENAI_DEPLOYMENT_NAME: AZURE_OPENAI_DEPLOYMENT_NAME = userdata.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if not AZURE_OPENAI_API_KEY: raise ValueError("Azure OpenAI APIキー (AZURE_OPENAI_API_KEY) が設定されていません。")
    if not AZURE_OPENAI_ENDPOINT: raise ValueError("Azure OpenAI エンドポイント (AZURE_OPENAI_ENDPOINT) が設定されていません。")
    if not OPENAI_API_VERSION: OPENAI_API_VERSION = "2023-07-01-preview" # デフォルトを設定することも可能
    if not AZURE_OPENAI_DEPLOYMENT_NAME: raise ValueError("Azure OpenAI デプロイメント名 (AZURE_OPENAI_DEPLOYMENT_NAME) が設定されていません。")

    os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
    os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
    os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION

# --- Google Cloud Vertex AI (Gemini) ---
elif LLM_PROVIDER == "google":
    PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT_ID") # .env 用に修正
    LOCATION = os.environ.get("GOOGLE_CLOUD_LOCATION")

    if IS_COLAB:
        if not PROJECT_ID: PROJECT_ID = userdata.get("GOOGLE_CLOUD_PROJECT_ID")
        if not LOCATION: LOCATION = userdata.get("GOOGLE_CLOUD_LOCATION") # 例: "us-central1"
        from google.colab import auth as google_auth
        google_auth.authenticate_user() # Vertex AI を使う場合は Colab での認証を推奨
    else: # Colab外の場合、.envから読み込んだ値で環境変数を設定
        if PROJECT_ID: os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID # Vertex AI SDKが参照する標準的な環境変数名
        if LOCATION: os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION

    if not PROJECT_ID: raise ValueError("Google Cloud Project ID が設定されていません。環境変数 GOOGLE_CLOUD_PROJECT_ID を設定するか、Colab環境の場合はシークレットに GOOGLE_CLOUD_PROJECT_ID を設定してください。")
    if not LOCATION: LOCATION = "us-central1" # デフォルトロケーション

# --- Google Gemini API (langchain-google-genai) ---
elif LLM_PROVIDER == "google_genai":
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY and IS_COLAB:
        GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("Google APIキーが設定されていません。環境変数 GOOGLE_API_KEY を設定するか、Colab環境の場合はシークレットに GOOGLE_API_KEY を設定してください。")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# --- Anthropic (Claude) ---
elif LLM_PROVIDER == "anthropic":
    ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY and IS_COLAB:
        ANTHROPIC_API_KEY = userdata.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY:
        raise ValueError("Anthropic APIキーが設定されていません。環境変数 ANTHROPIC_API_KEY を設定するか、Colab環境の場合はシークレットに ANTHROPIC_API_KEY を設定してください。")
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

# --- Amazon Bedrock (Claude) ---
elif LLM_PROVIDER == "bedrock":
    AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
    AWS_REGION_NAME = os.environ.get("AWS_REGION_NAME")

    if IS_COLAB: 
        if not AWS_ACCESS_KEY_ID: AWS_ACCESS_KEY_ID = userdata.get("AWS_ACCESS_KEY_ID")
        if not AWS_SECRET_ACCESS_KEY: AWS_SECRET_ACCESS_KEY = userdata.get("AWS_SECRET_ACCESS_KEY")
        if not AWS_REGION_NAME: AWS_REGION_NAME = userdata.get("AWS_REGION_NAME")

    if not AWS_REGION_NAME:
         raise ValueError("AWSリージョン名 (AWS_REGION_NAME) が設定されていません。Bedrock利用にはリージョン指定が必要です。")

    # 環境変数に設定 (boto3がこれらを自動で読み込む)
    if AWS_ACCESS_KEY_ID: os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
    if AWS_SECRET_ACCESS_KEY: os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
    os.environ["AWS_DEFAULT_REGION"] = AWS_REGION_NAME # boto3が参照する標準的なリージョン環境変数名
    os.environ["AWS_REGION"] = AWS_REGION_NAME # いくつかのライブラリはこちらを参照することもある

print(f"APIキー/環境変数の設定完了 (プロバイダー: {LLM_PROVIDER})")

### LLMクライアントの初期化

このセルは、上で選択・設定したLLMプロバイダーに基づいて、対応するLLMクライアントを初期化します。

In [ ]:
# === LLMクライアントの動的初期化 ===
llm = None

if LLM_PROVIDER == "openai":
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
elif LLM_PROVIDER == "azure":
    from langchain_openai import AzureChatOpenAI
    llm = AzureChatOpenAI(
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"), # 環境変数から取得
        openai_api_version=os.environ.get("OPENAI_API_VERSION"), # 環境変数から取得
        temperature=0,
    )
elif LLM_PROVIDER == "google":
    from langchain_google_vertexai import ChatVertexAI
    # PROJECT_ID, LOCATION は前のセルで環境変数に設定済みか、Colabの場合は直接利用
    llm = ChatVertexAI(model_name="gemini-2.0-flash", temperature=0, project=os.environ.get("GOOGLE_CLOUD_PROJECT"), location=os.environ.get("GOOGLE_CLOUD_LOCATION"))
elif LLM_PROVIDER == "google_genai":
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
elif LLM_PROVIDER == "anthropic":
    from langchain_anthropic import ChatAnthropic
    llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
elif LLM_PROVIDER == "bedrock":
    from langchain_aws import ChatBedrock # langchain_community.chat_models から langchain_aws に変更の可能性あり
    # AWS_REGION_NAME は前のセルで環境変数 AWS_DEFAULT_REGION に設定済み
    llm = ChatBedrock( # BedrockChat ではなく ChatBedrock が一般的
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        # region_name=os.environ.get("AWS_DEFAULT_REGION"), # 通常、boto3が環境変数から自動で読み込む
        model_kwargs={"temperature": 0},
    )
else:
    raise ValueError(
        f"Unsupported LLM_PROVIDER: {LLM_PROVIDER}. "
        "Please choose from 'openai', 'azure', 'google', 'google_genai', 'anthropic', or 'bedrock'."
    )

print(f"LLM Provider: {LLM_PROVIDER}")
if llm:
    print(f"LLM Client Type: {type(llm)}")
    # モデル名取得の試行を汎用的に
    model_attr = (
                 getattr(llm, 'model', None) or
                 getattr(llm, 'model_name', None) or
                 getattr(llm, 'model_id', None) or
                 (hasattr(llm, 'llm') and getattr(llm.llm, 'model', None)) # 一部のLLMクライアントのネスト構造に対応
    )
    if hasattr(llm, 'azure_deployment') and not model_attr: # Azure特有の属性
        model_attr = llm.azure_deployment
        
    if model_attr:
        print(f"LLM Model: {model_attr}")
    else:
        print("LLM Model: (Could not determine model name from client attributes)")


---

### ■ 問題001: 最小構成のLangGraphグラフの構築

LangGraphの最も基本的な構成要素である`StateGraph`と`State`を理解し、シンプルなグラフを構築してみましょう。この問題では、入力された文字列をそのまま出力するだけの、単一のノードを持つグラフを作成します。

*   **学習内容:** この問題では、`StateGraph`、`TypedDict`を用いた`State`の定義、`add_node`、`set_entry_point`、`add_edge`、`END`といったLangGraphの最も基本的なAPIを学びます。また、`Annotated`と`add_messages`を使ってメッセージ履歴を管理する方法も理解します。


In [ ]:
# 解答欄001

from ____ import ____, ____
from ____.____ import ____, ____
from ____.____.____ import ____

# --- 状態定義 (State) ---
class ____(____):
    ____: ____[____, ____]

# --- ノード定義 (Nodes) ---
def ____(____: GraphState):
    print(f'simple_node: {____[____][____].____}')
    return {____: [____[____][____]]}

# --- グラフ構築 (Graph) ---
____ = StateGraph(GraphState)

# ノードの追加
workflow.add_node(____, simple_node)

# エントリポイントの設定
workflow.set_entry_point(____)

# 終了ポイントの設定
workflow.add_edge(____, ____)

# グラフのコンパイル
____ = workflow.compile()

# --- グラフの実行と結果表示 ---
____ = {"messages": [("user", "Hello, LangGraph!")]}

# 最終結果の確認
____ = graph.invoke(inputs)
print(f"最終的な応答: {final_state['messages'][-1].content}")

<details><summary>解答001</summary>

``````python
# 解答001

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    # グラフの状態を保持する辞書
    # ここでは、入力メッセージを保持する
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def simple_node(state: GraphState):
    # 入力されたメッセージをそのまま返すノード
    print(f'simple_node: {state["messages"][-1].content}')
    return {"messages": [state["messages"][-1]]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("simple_node", simple_node)

# エントリポイントの設定
workflow.set_entry_point("simple_node")

# 終了ポイントの設定
workflow.add_edge("simple_node", END)

# グラフのコンパイル
graph = workflow.compile()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [("user", "Hello, LangGraph!")]}

# 最終結果の確認
final_state = graph.invoke(inputs)
print(f"最終的な応答: {final_state['messages'][-1].content}")
``````
</details>

<details><summary>解説001</summary>

#### この問題のポイント

*   **コード解説:**
    *   `GraphState`は、グラフ全体で共有される状態を定義します。`TypedDict`を使うことで、状態のスキーマを明確にできます。`messages: Annotated[list, add_messages]`は、LangChainのメッセージ形式のリストを状態として持ち、新しいメッセージが追加されるたびに自動的にリストの末尾に追加されるように設定しています。
    *   `simple_node`関数は、グラフのノードとして機能します。`state`引数として現在のグラフの状態を受け取り、新しい状態を辞書として返します。ここでは、入力された最後のメッセージをそのまま返しています。
    *   `StateGraph(GraphState)`でグラフのインスタンスを作成し、`GraphState`で定義した状態スキーマを渡します。
    *   `workflow.add_node("simple_node", simple_node)`で、`simple_node`関数を`simple_node`という名前のノードとしてグラフに追加します。
    *   `workflow.set_entry_point("simple_node")`は、グラフの実行が開始される最初のノードを指定します。
    *   `workflow.add_edge("simple_node", END)`は、`simple_node`の実行が完了したらグラフを終了することを示します。`END`はLangGraphが提供する特別な終了ノードです。
    *   `graph = workflow.compile()`で、定義したワークフローを実行可能なアプリケーションにコンパイルします。
    *   `graph.stream(inputs)`は、グラフの実行過程をストリーミングで受け取ることができます。`graph.invoke(inputs)`は、グラフの実行が完了した最終状態を返します。
---
</details>

### ■ 問題002: グラフの可視化

問題001で構築した最小構成のグラフの構造を、視覚的に確認する方法を学びましょう。

*   **学習内容:** `graph.get_graph().draw_png()` を使用して、コンパイル済みのLangGraphグラフ構造をPNG画像として描画し、Jupyter Notebook上に表示する方法を学びます。これにより、グラフのノードとエッジの接続関係を直感的に理解できるようになります。

In [ ]:
# 解答欄002

# 問題001のコードを再掲（このセルでグラフを定義・コンパイルします）
from typing import ____, ____
from langgraph.____ import ____, ____
from langgraph.graph.____ import ____
from langchain_core.____ import ____

# --- 状態定義 (State) ---
class ____(____):
    ____: ____[list, add_messages]

# --- ノード定義 (Nodes) ---
def ____(state: GraphState):
    # このノードは状態を更新せず、最後のメッセージをログに出力するだけ
    print(f"simple_node: Received message -> {____[____][____].____}")
    # LangGraphでは、ノードがNoneまたは空の辞書を返すと、状態は更新されない
    ____

# --- グラフ構築 (Graph) ---
____ = StateGraph(GraphState)
____.add_node("simple_node", simple_node)
____.set_entry_point(____)
____.add_edge(____, ____)
____ = workflow.compile()

In [ ]:
# --- グラフの可視化 ---
from IPython.____ import ____, ____

try:
    ____ = graph.get_graph().draw_png()
    ____(Image(png_data))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

In [ ]:
# --- グラフの動作確認 ---
# 可視化したグラフが問題001と同様に動作することを確認します。
try:
    inputs = {"messages": [HumanMessage(content="Hello, this is a test.")]}
    final_state = graph.invoke(inputs)
    print("\nグラフの実行が完了しました。")
    print(f"最終的なmessagesの状態: {final_state['messages']}")
except NameError:
    print("グラフが定義されていません。前のセルを先に実行してください。")

<details><summary>解答002</summary>

``````python
# 解答002

# 問題001のコードを再掲（このセルでグラフを定義・コンパイルします）
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def simple_node(state: GraphState):
    # このノードは状態を更新せず、最後のメッセージをログに出力するだけ
    print(f"simple_node: Received message -> {state['messages'][-1].content}")
    # LangGraphでは、ノードがNoneまたは空の辞書を返すと、状態は更新されない
    return

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)
workflow.add_node("simple_node", simple_node)
workflow.set_entry_point("simple_node")
workflow.add_edge("simple_node", END)
graph = workflow.compile()

# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    png_data = graph.get_graph().draw_png()
    display(Image(png_data))
    print("グラフが正常に可視化されました。")
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

# --- グラフの動作確認 ---
# 可視化したグラフが問題001と同様に動作することを確認します。
try:
    inputs = {"messages": [HumanMessage(content="Hello, this is a test.")]}
    final_state = graph.invoke(inputs)
    print("\nグラフの実行が完了しました。")
    print(f"最終的なmessagesの状態: {final_state['messages']}")
except NameError:
    print("グラフが定義されていません。前のセルを先に実行してください。")
``````
</details>

<details><summary>解説002</summary>

#### この問題のポイント

*   **コード解説:**
    *   この問題では、まず問題001で作成したグラフ定義のコードを再利用して、`graph`オブジェクトを準備します。
    *   グラフを可視化するための中心的なメソッドが `graph.get_graph().draw_png()` です。
        *   `graph.get_graph()`: コンパイル済みの`graph`オブジェクトから、可視化や解析が可能な内部グラフ表現を取得します。
        *   `.draw_png()`: 取得したグラフ表現をPNG形式の画像データ（バイト列）として描画します。
    *   `IPython.display.Image` は、画像データをJupyter NotebookなどのIPython環境で表示可能なオブジェクトに変換します。
    *   `IPython.display.display()` 関数を使って、`Image`オブジェクトをセルに表示します。
    *   **重要:** `.draw_png()` メソッドを使用するには、**Graphviz**というグラフ可視化ソフトウェアがシステムにインストールされている必要があります。また、Pythonライブラリの`pygraphviz`も必要です（これらは準備セクションでインストール済みです）。もし`ExecutableNotFound`のようなエラーが出る場合は、Graphviz本体がOSに正しくインストールされ、パスが通っているかを確認してください。

*   **なぜ可視化が重要か:**
    *   グラフが単純なうちはコードを読むだけで構造を理解できますが、ノードやエッジが増え、特に条件分岐やループが絡んでくると、全体の流れを把握するのが難しくなります。
    *   グラフを可視化することで、設計した通りの構造になっているかを一目で確認でき、意図しない接続やループのデバッグに非常に役立ちます。

---
</details>

### ■ 問題003: 複数のノードを持つシーケンシャルグラフの構築

前の問題で学んだ基本的なグラフ構築に加えて、複数のノードを直列に接続し、データがノード間をどのように流れるかを理解しましょう。ここでは、入力された文字列を加工する2つのノード（例：大文字化、逆順化）を持つグラフを作成します。

*   **学習内容:** 複数のノードを`add_edge`で直列に接続する方法と、ノード間で状態がどのように引き継がれるかを学びます。`HumanMessage`と`AIMessage`を使って、メッセージの送信元を明示する方法も理解します。

In [ ]:
# 解答欄003

from ____ import ____, ____
from ____.____ import ____, ____
from ____.____.____ import ____
from ____.____.____ import ____, ____

# --- 状態定義 (State) ---
class ____(____):
    ____: ____[____, ____]

# --- ノード定義 (Nodes) ---
def ____(state: GraphState):
    last_message_content = ____[____][____].____
    print(f"uppercase_node: {last_message_content}")
    return {____: [____(content=last_message_content.upper())]}

def ____(state: GraphState):
    last_message_content = ____[____][____].____
    print(f"reverse_node: {last_message_content}")
    return {____: [____(content=last_message_content[::-1])]}

# --- グラフ構築 (Graph) ---
____ = ____(GraphState)

# ノードの追加
____.____("uppercase", uppercase_node)
____.____("reverse", reverse_node)

# エントリポイントの設定
____.____("uppercase")

# エッジの追加 (直列接続)
____.____("uppercase", "reverse")
____.____("reverse", END)

# グラフのコンパイル
____ = ____.compile()

In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

In [ ]:
# --- グラフの実行と結果表示 ---
inputs = {"messages": [HumanMessage(content="Hello LangGraph")]}

final_state = graph.invoke(inputs)
print(f"最終的な応答: {final_state['messages'][-1].content}")

<details><summary>解答003</summary>

``````python
# 解答003

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def uppercase_node(state: GraphState):
    # 最新のメッセージを大文字に変換するノード
    last_message_content = state["messages"][-1].content
    print(f"uppercase_node: {last_message_content}")
    return {"messages": [AIMessage(content=last_message_content.upper())]}

def reverse_node(state: GraphState):
    # 最新のメッセージを逆順にするノード
    last_message_content = state["messages"][-1].content
    print(f"reverse_node: {last_message_content}")
    return {"messages": [AIMessage(content=last_message_content[::-1])]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("uppercase", uppercase_node)
workflow.add_node("reverse", reverse_node)

# エントリポイントの設定
workflow.set_entry_point("uppercase")

# エッジの追加 (直列接続)
workflow.add_edge("uppercase", "reverse")
workflow.add_edge("reverse", END)

# グラフのコンパイル
graph = workflow.compile()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [HumanMessage(content="Hello LangGraph")]}

final_state = graph.invoke(inputs)
print(f"最終的な応答: {final_state['messages'][-1].content}")
``````
</details>

<details><summary>解説003</summary>

#### この問題のポイント

*   **コード解説:**
    *   `uppercase_node`と`reverse_node`は、それぞれ入力メッセージを大文字化、逆順化する処理を行います。重要なのは、各ノードが新しい`AIMessage`を作成して状態に返す点です。これにより、次のノードは前のノードの処理結果を`state["messages"][-1]`で取得できます。
    *   `workflow.add_edge("uppercase", "reverse")`は、`uppercase`ノードの実行が完了したら、次に`reverse`ノードを実行するように指示します。このようにして、処理の流れを定義します。
    *   入力メッセージを`HumanMessage`として渡すことで、ユーザーからの入力であることを明示しています。ノードからの出力は`AIMessage`として返され、メッセージ履歴にAIの応答として記録されます。
---
</details>

### ■ 問題004: グラフ内でのLLMの利用（シンプルなチャットボット）

LangGraphのノード内で大規模言語モデル（LLM）を呼び出す方法を学び、シンプルなチャットボットを構築しましょう。ここでは、ユーザーからの入力に対してLLMが応答を生成し、その応答を返すグラフを作成します。

In [ ]:
# 解答欄004
from typing import ____, ____
from langgraph.____ import ____, ____
from langgraph.____.____ import ____
from langchain_core.____ import HumanMessage, AIMessage
import ____

# ノートブック冒頭で`llm`変数が初期化されている前提
# (from langchain_openai import ChatOpenAI や llm = ChatOpenAI(...) といった行はここには不要)

# --- 状態定義 (State) ---
class ____(TypedDict):
    ____: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def ____(state: GraphState):
    # LLMを呼び出し、応答を生成するノード
    # ノートブック冒頭で初期化された共通の `llm` 変数を使用します。
    print(f"llm_node: Calling LLM with messages: {____[____]}")
    ____ = llm.invoke(state["messages"]) 
    print(f"llm_node: LLM response: {____.____}")
    return {____: [response]} # responseはAIMessageオブジェクトを期待 (ここは歯抜けにしない)

# --- グラフ構築 (Graph) ---
____ = StateGraph(GraphState)

# ノードの追加
____.____("llm_responder", llm_node)

# エントリポイントの設定
____.____("llm_responder")

# 終了ポイントの設定
____.____("llm_responder", END)

# グラフのコンパイル
____ = ____.compile()

In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

In [ ]:
# --- グラフの実行と結果表示 ---
print("\n--- チャットボットのテスト ---")
# 最初のメッセージはHumanMessageであると想定
inputs = {"messages": [HumanMessage(content="こんにちは、あなたの名前は何ですか？")]}

final_state = graph.invoke(inputs)
print(f"最終的な応答: {final_state['messages'][-1].content}")

print("\n--- 別の質問 ---")
inputs2 = {"messages": [HumanMessage(content="今日の天気は？")]}

final_state2 = graph.invoke(inputs2)
print(f"最終的な応答: {final_state2['messages'][-1].content}")


<details><summary>解答004</summary>

``````python
# 解答004

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
import os # osはAPIキー設定のコメントアウト部分で使われているので残しても良いが、直接は不要になる

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def llm_node(state: GraphState):
    # LLMを呼び出し、応答を生成するノード
    # ノートブック冒頭で初期化された共通の `llm` 変数を使用します。
    print(f"llm_node: Calling LLM with messages: {state['messages']}")
    response = llm.invoke(state["messages"]) # 共通llmを使用
    print(f"llm_node: LLM response: {response.content}")
    return {"messages": [response]} # responseはAIMessageオブジェクトを期待

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("llm_responder", llm_node)

# エントリポイントの設定
workflow.set_entry_point("llm_responder")

# 終了ポイントの設定
workflow.add_edge("llm_responder", END)

# グラフのコンパイル
graph = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- チャットボットのテスト ---")
# 最初のメッセージはHumanMessageであると想定
inputs = {"messages": [HumanMessage(content="こんにちは、あなたの名前は何ですか？")]}

final_state = graph.invoke(inputs)
print(f"最終的な応答: {final_state['messages'][-1].content}")

print("\n--- 別の質問 ---")
inputs2 = {"messages": [HumanMessage(content="今日の天気は？")]}

final_state2 = graph.invoke(inputs2)
print(f"最終的な応答: {final_state2['messages'][-1].content}")
``````
</details>

<details><summary>解説004</summary>

このノートブックでは、様々なLLMプラットフォーム（OpenAI, Azure OpenAI, Google Cloud Vertex AI, Google Gemini (Gemini API), Anthropic Claude, Amazon Bedrockなど）を簡単に切り替えて試せるように設計されています。
ノートブックの冒頭にある `LLM_PROVIDER` 変数で使用したいLLMを選択し、対応するAPIキーや環境変数を設定するだけで、この問題を含む全てのLLM呼び出し箇所で選択したLLMが利用されます。
選択した `LLM_PROVIDER` に応じて、必要なAPIキーが設定されているか（環境変数またはGoogle Colabのシークレット経由）、ノートブック起動時にチェックされます。

ここでは、ノートブックの先頭で設定・初期化された共通の `llm` 変数を使用して、LLMに質問をしています。
`llm.invoke()` という統一されたインターフェースで、どのLLMプロバイダーを利用しているかに関わらず、同じようにLLMを呼び出すことができます。
これにより、特定のLLMサービスに依存しない、より汎用的なコードを作成するメリットを手軽に体験できます。

もしエラーが発生した場合は、ノートブック冒頭の `LLM_PROVIDER` の設定、および選択したプロバイダーに応じたAPIキーや環境変数の設定（例: `OPENAI_API_KEY`, `GOOGLE_API_KEY`, `AZURE_OPENAI_ENDPOINT`など）が正しく行われているかを確認してください。
各プロバイダー固有の設定項目（例えばVertex AIのProject ID、AzureのDeployment Name、Bedrockのリージョンなど）も見直してください。
プロバイダーによっては、`pip install` コマンドで対応するライブラリ (例: `langchain-google-genai`) がインストールされているかも確認点です。

</details>

### ■ 問題005: 状態の更新 - 特定キーの値を上書きする

`add_messages` によるメッセージ履歴の追加だけでなく、グラフの状態(`State`)内の特定のキーの値を直接更新する方法を学びましょう。ここでは、カウンター値を保持する状態キーを定義し、ノードでその値をインクリメントするグラフを作成します。

*   **学習内容:** `TypedDict`で定義する状態クラスに、`messages`以外のカスタムキー（ここでは`counter: int`）を追加し、ノード関数内でその値を直接読み書きする方法を学びます。これにより、メッセージ履歴だけでなく、数値や文字列、ブール値など、より多様なデータをグラフ全体で管理できるようになります。


In [ ]:
# 解答欄005

from ____ import ____, ____
from ____.____ import ____, ____
from ____.____.____ import ____
from ____.____.____ import ____

# --- 状態定義 (State) ---
class ____(____):
    ____: ____[____, ____]
    ____: ____ 

# --- ノード定義 (Nodes) ---
def ____(____: CounterState):
    # counterの値を1増やすノード
    ____ = ____.get("counter", 0) 
    ____ = current_count + 1
    print(f"increment_counter: Current count: {current_count}, New count: {new_count}")
    return {____: new_count, ____: [HumanMessage(content=f"Counter incremented to {new_count}")]}

def ____(state: CounterState):
    # counterの最終値を表示するノード (実際にはmessagesに追加されたもので確認)
    print(f"display_count: Final counter value is {state['counter']}")
    # このノードは状態を更新しないが、メッセージを追加しても良い
    return {"messages": [HumanMessage(content=f"Final count: {state['counter']}")]}

# --- グラフ構築 (Graph) ---
____ = ____(CounterState)

# ノードの追加
____.____("increment", increment_counter)
____.____("display", display_count)

# エントリポイントの設定
____.____("increment")

# エッジの追加
____.____("increment", "display")
____.____("display", END)

# グラフのコンパイル
____ = ____.compile()

In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

In [ ]:
# --- グラフの実行と結果表示 ---
print("\n--- カウンターテスト (初期値0から) ---")
inputs = {"messages": [HumanMessage(content="Start counting")], "counter": 0} # 初期カウンター値を設定

final_state = graph.invoke(inputs)
print(f"最終的な応答: {final_state['messages'][-1].content}")

print("\n--- カウンターテスト (初期値5から) ---")
inputs_2 = {"messages": [HumanMessage(content="Start counting from 5")], "counter": 5} # 初期カウンター値を設定

final_state_2 = graph.invoke(inputs_2)
print(f"最終的な応答: {final_state_2['messages'][-1].content}")


<details><summary>解答005</summary>

``````python
# 解答005

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage

# --- 状態定義 (State) ---
class CounterState(TypedDict):
    messages: Annotated[list, add_messages]
    counter: int # 新しくカウンター用の状態キーを定義

# --- ノード定義 (Nodes) ---
def increment_counter(state: CounterState):
    # counterの値を1増やすノード
    current_count = state.get("counter", 0) # stateからcounterの値を取得、なければ0
    new_count = current_count + 1
    print(f"increment_counter: Current count: {current_count}, New count: {new_count}")
    return {"counter": new_count, "messages": [HumanMessage(content=f"Counter incremented to {new_count}")]}

def display_count(state: CounterState):
    # counterの最終値を表示するノード (実際にはmessagesに追加されたもので確認)
    print(f"display_count: Final counter value is {state['counter']}")
    # このノードは状態を更新しないが、メッセージを追加しても良い
    return {"messages": [HumanMessage(content=f"Final count: {state['counter']}")]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(CounterState)

# ノードの追加
workflow.add_node("increment", increment_counter)
workflow.add_node("display", display_count)

# エントリポイントの設定
workflow.set_entry_point("increment")

# エッジの追加
workflow.add_edge("increment", "display")
workflow.add_edge("display", END)

# グラフのコンパイル
graph = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- カウンターテスト (初期値0から) ---")
inputs = {"messages": [HumanMessage(content="Start counting")], "counter": 0} # 初期カウンター値を設定

final_state = graph.invoke(inputs)
print(f"最終的な応答: {final_state['messages'][-1].content}")

print("\n--- カウンターテスト (初期値5から) ---")
inputs_2 = {"messages": [HumanMessage(content="Start counting from 5")], "counter": 5} # 初期カウンター値を設定

final_state_2 = graph.invoke(inputs_2)
print(f"最終的な応答: {final_state_2['messages'][-1].content}")
``````
</details>

<details><summary>解説005</summary>

#### この問題のポイント

*   **コード解説:**
    *   `CounterState`に`counter: int`を追加しました。これにより、グラフの状態はメッセージリストと整数型のカウンターを持つことになります。
    *   `increment_counter`ノードでは、`state.get("counter", 0)`を使って現在のカウンター値を取得しています。`.get()`メソッドを使うことで、キーが存在しない場合のデフォルト値を指定できます（ここでは初回実行時を想定して0）。その後、値をインクリメントし、更新後の値を`{"counter": new_count}`という辞書形式で返しています。LangGraphは、ノードが返す辞書のキーに基づいて対応する状態を更新します。
    *   `messages`キーも同時に返すことで、状態更新のログや情報をメッセージ履歴に残すことができます。
    *   グラフ実行時 (`graph.invoke`や`graph.stream`) に、`inputs`辞書に`"counter": 0`（または任意の値）を含めることで、`counter`キーの初期値を設定できます。
    *   このように、ノードは状態の一部または全部を更新する辞書を返すことで、グラフの状態を変化させます。`add_messages`はメッセージリストの更新に特化した便利な方法ですが、他のキーは直接値を指定して更新します。
---
</details>

### ■ 問題006: 状態の更新 - 複数のキーを一度に更新する

一つのノードから、状態(`State`)の複数のキーを同時に更新する方法を学びましょう。ここでは、ユーザーからのメッセージ内容に応じて、応答メッセージと共に「応答タイプ」という別の状態キーも更新するグラフを作成します。

*   **学習内容:** 一つのノード関数から返す辞書に複数のキーと値のペアを含めることで、グラフの状態(`State`)の複数の属性を一度に更新する方法を学びます。また、`typing.Literal`を使って、状態キーが取りうる値を制限する方法も示します。


In [ ]:
# 解答欄006
from ____ import ____, ____, ____
from ____.____ import ____, ____
from ____.____.____ import ____
from ____.____.____ import ____, ____

# --- 状態定義 (State) ---
____ = Literal["greeting", "question", "other", "unknown"]

class ____(____):
    ____: Annotated[list, add_messages]
    ____: ResponseType
    ____: str 

# --- ノード定義 (Nodes) ---
def ____(state: MultiUpdateState):
    user_message = ____[____][____].content.lower()
    response_text = ""
    resp_type: ResponseType = "unknown"

    if "こんにちは" in user_message or "こんばんは" in user_message:
        response_text = "こんにちは！何かお手伝いできますか？"
        resp_type = "greeting" 
    elif "?" in user_message or "教えて" in user_message:
        response_text = "ご質問ありがとうございます。それについては現在調べています。"
        resp_type = "question" 
    else:
        response_text = "メッセージありがとうございます。"
        resp_type = "other" 
    
    print(f"process_input: User: '{user_message}', AI: '{response_text}', Type: '{resp_type}'")
    # 複数のキーを同時に更新して返す
    return { 
        "messages": [____(content=response_text)],
        "response_type": ____,
        "last_user_message": ____
    }

# --- グラフ構築 (Graph) ---
____ = ____(MultiUpdateState)

____.____("processor", process_input)
____.____("processor")
____.____("processor", END)

____ = ____.compile()

In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

In [ ]:
# --- グラフの実行と結果表示 ---
test_inputs = [
    {"messages": [HumanMessage(content="こんにちは")]},
    {"messages": [HumanMessage(content="LangGraphについて教えてください")]},
    {"messages": [HumanMessage(content="今日の天気は晴れですね")]}
]

for i, inputs in enumerate(test_inputs):
    print(f"\n--- テスト実行 {i+1} ---")

    final_state = graph.invoke(inputs, {"recursion_limit": 3})
    print(f"最終的な応答: {final_state['messages'][-1].content}")
    assert "response_type" in final_state
    assert "last_user_message" in final_state
    print(f"Response Type: {final_state['response_type']}")
    print(f"Last User Message: {final_state['last_user_message']}")


<details><summary>解答006</summary>

``````python
# 解答006
from typing import TypedDict, Annotated, Literal
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
ResponseType = Literal["greeting", "question", "other", "unknown"]

class MultiUpdateState(TypedDict):
    messages: Annotated[list, add_messages]
    response_type: ResponseType # 応答の種類を保持するキー
    last_user_message: str # 最後に入力されたユーザーメッセージ

# --- ノード定義 (Nodes) ---
def process_input(state: MultiUpdateState):
    user_message = state["messages"][-1].content.lower()
    response_text = ""
    resp_type: ResponseType = "unknown"

    if "こんにちは" in user_message or "こんばんは" in user_message:
        response_text = "こんにちは！何かお手伝いできますか？"
        resp_type = "greeting"
    elif "?" in user_message or "教えて" in user_message:
        response_text = "ご質問ありがとうございます。それについては現在調べています。"
        resp_type = "question"
    else:
        response_text = "メッセージありがとうございます。"
        resp_type = "other"
    
    print(f"process_input: User: '{user_message}', AI: '{response_text}', Type: '{resp_type}'")
    # 複数のキーを同時に更新して返す
    return {
        "messages": [AIMessage(content=response_text)],
        "response_type": resp_type,
        "last_user_message": user_message # 元のユーザーメッセージを保存
    }

# --- グラフ構築 (Graph) ---
workflow = StateGraph(MultiUpdateState)

workflow.add_node("processor", process_input)
workflow.set_entry_point("processor")
workflow.add_edge("processor", END)

graph = workflow.compile()

# --- グラフの実行と結果表示 ---
test_inputs = [
    {"messages": [HumanMessage(content="こんにちは")]},
    {"messages": [HumanMessage(content="LangGraphについて教えてください")]},
    {"messages": [HumanMessage(content="今日の天気は晴れですね")]}
]

for i, inputs in enumerate(test_inputs):
    print(f"\n--- テスト実行 {i+1} ---")
    # 初期状態としてresponse_typeやlast_user_messageを渡すことも可能だが、
    # この問題ではノード内でこれらが設定されることを確認する
    initial_state = inputs.copy()
    # 必要であれば、初期値を設定
    # initial_state.setdefault("response_type", "unknown") 
    # initial_state.setdefault("last_user_message", "")

    final_state = graph.invoke(initial_state, {"recursion_limit": 3})
    print(f"最終的な応答: {final_state['messages'][-1].content}")
    assert "response_type" in final_state
    assert "last_user_message" in final_state
    print(f"Response Type: {final_state['response_type']}")
    print(f"Last User Message: {final_state['last_user_message']}")
``````
</details>

<details><summary>解説006</summary>

#### この問題のポイント

*   **コード解説:**
    *   `MultiUpdateState`に、`response_type: ResponseType` と `last_user_message: str` という2つの新しいキーを追加しました。`ResponseType`は`Literal["greeting", "question", "other", "unknown"]`で定義され、`response_type`キーがこれらの文字列のうちのいずれかの値を取ることを示します（型ヒントであり、実行時の厳密な強制ではありませんが、開発時の可読性や静的解析に役立ちます）。
    *   `process_input`ノードは、ユーザーのメッセージ内容に基づいて応答メッセージを生成し、それと同時に`response_type`（挨拶、質問、その他など）と`last_user_message`（処理対象となった元のユーザーメッセージ）も決定します。
    *   ノードが返す辞書は `{"messages": ..., "response_type": ..., "last_user_message": ...}` のようになります。LangGraphは、この辞書に含まれる各キーに対応する状態を更新します。
    *   実行時には、`messages`キーだけでなく、`response_type`と`last_user_message`も最終状態に含まれていることを確認できます。
    *   このように、ノードはグラフの状態を柔軟に更新する役割を担います。返す辞書に含めるキーと値によって、どの状態属性をどのように変更するかを制御できます。
---
</details>

### ■ 問題007: `END` 以外の終了ノードの指定（概念理解）

LangGraphでは、グラフの終点は通常、特別な `END` ノードに接続することで示されます。しかし、概念的には、あるノードが処理の最終ステップであり、それ以上後続のノードが存在しない場合、そのノードが事実上の「終了ノード」として機能すると考えることもできます。この問題では、特定のノードを実行した後、明示的に `END` に接続せず、グラフがそこで停止することを確認します。ただし、LangGraphのベストプラクティスとしては、可能な限り `END` を使用することが推奨されます。

In [ ]:
# 解答欄007
from ____ import ____, ____
from ____.____ import ____, ____ 
from ____.____.____ import ____
from ____.____.____ import ____, ____

# --- 状態定義 (State) ---
class ____(____):
    ____: ____[____, ____]
    ____: str

# --- ノード定義 (Nodes) ---
def ____(state: FinalNodeState):
    print("start_process: Process started.")
    return {____: "Processing", ____: [AIMessage(content="Process initiated.")]}

def ____(state: FinalNodeState):
    # このノードが処理の最後とする
    ____ = "Process completed at final_processing_node."
    print(f"final_processing_node: {final_message}")
    return {____: "Completed", ____: [AIMessage(content=final_message)]}

# --- グラフ構築 (Graph) ---
____ = ____(FinalNodeState)

____.____("start", start_process)
____.____("final_step", final_processing_node)

____.____("start")

# startノードからfinal_stepノードへのエッジ
____.____("start", "final_step")

# final_stepノードからENDへのエッジを意図的に作成しない
# ____.____("final_step", END) 

# グラフのコンパイル
# check_interruptions=True をつけると、ENDに到達しない場合にエラーになるため、
# この例では明示的にENDに繋がないことを示すために compile() の引数なし、
# または check_interruptions=False (デフォルト) を利用します。
# Langfuseなどのトレーシングツールと連携する場合、ENDに到達しないとトレースが終了しないことがあるため注意。
____ = ____.____()

In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

In [ ]:
# --- グラフの実行と結果表示 ---
print("\n--- 最終ノードテスト ---")
inputs = {"messages": [HumanMessage(content="Begin process")], "status": "Initial"}
final_state_from_stream = None

print("Streaming execution:")
for chunk in graph.stream(inputs, {"recursion_limit": 5}):
    print(f"  Stream chunk: {chunk}")
    final_state_from_stream = chunk # Capture the last chunk which contains the state of the last executed node

print(f"Final State from stream: {final_state_from_stream}")

# invokeの挙動確認
invoked_state = None
try:
    invoked_state = graph.invoke(inputs, {"recursion_limit": 5})
    print(f"最終的な応答 (invoke): {invoked_state['messages'][-1].content}")
except Exception as e:
    print(f"Invoke call resulted in an error or unexpected behavior: {e}")
    print("This might be expected if the graph doesn't explicitly reach END.")

assert final_state_from_stream is not None, "Final state was not captured from stream"
final_node_key = list(final_state_from_stream.keys())[0] # Get the key of the last node's state
assert final_state_from_stream[final_node_key]["status"] == "Completed"
assert "Process completed at final_processing_node." in final_state_from_stream[final_node_key]["messages"][-1].content
print("Assertion for final_state_from_stream passed.")

if invoked_state:
    print(f"Invoked state status: {invoked_state.get('status')}")

try:
    display(Image(graph.get_graph().draw_png()))
    print("Graph visualized. Note that 'final_step' does not point to END.")
except Exception as e:
    print(f"Graph visualization failed: {e}")


<details><summary>解答007</summary>

``````python
# 解答007
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class FinalNodeState(TypedDict):
    messages: Annotated[list, add_messages]
    status: str

# --- ノード定義 (Nodes) ---
def start_process(state: FinalNodeState):
    print("start_process: Process started.")
    return {"status": "Processing", "messages": [AIMessage(content="Process initiated.")]}

def final_processing_node(state: FinalNodeState):
    # このノードが処理の最後とする
    final_message = "Process completed at final_processing_node."
    print(f"final_processing_node: {final_message}")
    return {"status": "Completed", "messages": [AIMessage(content=final_message)]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(FinalNodeState)

workflow.add_node("start", start_process)
workflow.add_node("final_step", final_processing_node)

workflow.set_entry_point("start")

# startノードからfinal_stepノードへのエッジ
workflow.add_edge("start", "final_step")

# final_stepノードからENDへのエッジを意図的に作成しない
# workflow.add_edge("final_step", END) # ← コメントアウトまたは削除

# グラフのコンパイル
graph = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- 最終ノードテスト ---")
inputs = {"messages": [HumanMessage(content="Begin process")], "status": "Initial"}
final_state_from_stream = None

print("Streaming execution:")
for chunk in graph.stream(inputs, {"recursion_limit": 5}):
    print(f"  Stream chunk: {chunk}")
    final_state_from_stream = chunk

print(f"Final State from stream: {final_state_from_stream}")

invoked_state = None
try:
    invoked_state = graph.invoke(inputs, {"recursion_limit": 5})
    print(f"最終的な応答 (invoke): {invoked_state['messages'][-1].content}")
except Exception as e:
    print(f"Invoke call resulted in an error or unexpected behavior: {e}")
    print("This might be expected if the graph doesn't explicitly reach END.")

assert final_state_from_stream is not None, "Final state was not captured from stream"
final_node_key = list(final_state_from_stream.keys())[0]
assert final_state_from_stream[final_node_key]["status"] == "Completed"
assert "Process completed at final_processing_node." in final_state_from_stream[final_node_key]["messages"][-1].content
print("Assertion for final_state_from_stream passed.")

if invoked_state:
    print(f"Invoked state status: {invoked_state.get('status')}")

try:
    display(Image(graph.get_graph().draw_png()))
    print("Graph visualized. Note that 'final_step' does not point to END.")
except Exception as e:
    print(f"Graph visualization failed: {e}")
``````
</details>

<details><summary>解説007</summary>

#### この問題のポイント
*   **学習内容:** グラフの終点として必ずしも `END` を明示的に指定する必要はなく、あるノードから先に遷移するエッジがなければ、そのノードの処理が終わった時点でグラフの実行が停止し、その時点での状態が最終状態となることを理解します。ただし、これはLangGraphの挙動の一つであり、デバッグや可視化、他のツールとの連携（例: Langfuse）を考慮すると、可能な限りグラフの終点を `END` に接続することが推奨されます。
*   **コード解説:**
    *   `final_processing_node`を作成し、このノードから `END` へのエッジ（`workflow.add_edge("final_step", END)`）を定義していません。
    *   `graph.stream()` を使ってグラフを実行すると、`final_processing_node` が実行された後、それ以上進むべきノードがないため、処理が停止します。`stream()` の最後の出力（この場合は `final_processing_node` の出力）が、その実行における最終的な状態を示します。
    *   `graph.invoke()` の場合、グラフが明示的に `END` に到達しないと、バージョンや設定によってはエラーが発生したり、予期しない挙動をしたりする可能性があります。一般的に `invoke()` はグラフが `END` に到達し、完全な最終状態が確定することを期待します。この問題では、主に `stream()` での挙動を確認し、`invoke()` は参考として示しています。
    *   可視化すると、`final_step` ノードから `END` (または他のノード) への矢印がないことが確認できます。
*   **重要な注意点:**
    *   **`END` の使用推奨:** LangGraphでは、グラフの論理的な終了点を明確にするために `END` を使用することが強く推奨されます。これにより、グラフの構造が理解しやすくなり、デバッグも容易になります。また、LangSmith/Langfuseのようなトレースツールは、`END` への到達をもって一連の処理の完了とみなすことが多いため、連携時にも重要です。
    *   **`compile(check_interruptions=True)`:** グラフをコンパイルする際に `check_interruptions=True` を指定すると、中断（Interrupt）が発生しない限り、グラフが必ず `END` に到達することを強制できます。`END` に到達しないパスがある場合、コンパイル時または実行時にエラーが発生します。
    *   この問題は、`END` を使わない場合の挙動を理解するためのものであり、実際の開発では `END` を適切に配置する設計を心がけてください。
---
</details>

### ■ 問題008: LLMノードと非LLMノードの連携強化

LLM（大規模言語モデル）を組み込んだノードと、LLM以外の処理を行うノード（例: 文字列操作、データ抽出など）を連携させる方法を学びましょう。ここでは、LLMに質問を投げて得られた応答（文字列）から、特定の情報を抽出・加工して状態を更新する、より実践的なグラフを作成します。

*   **学習内容:** LLMを呼び出すノードと、その出力を処理する非LLMノードを組み合わせることで、より高度な情報処理パイプラインを構築する方法を学びます。具体的には、LLMの自然言語応答から正規表現などを用いて構造化された情報を抽出し、グラフの状態を更新します。


In [ ]:
# 解答欄008
from ____ import ____, ____
import ____ 
from ____.____ import ____, ____
from ____.____.____ import ____
from ____.____.____ import ____, ____

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class ____(____):
    ____: ____[____, ____]
    ____: str 
    ____: str 
    ____: str | ____ 

# --- ノード定義 (Nodes) ---
def ____(state: ExtractionState):
    # ユーザーの質問を状態に保存
    ____ = state["messages"][-1].content
    print(f"get_user_question: Received question: '{last_message_content}'")
    return {____: last_message_content}

def ____(state: ExtractionState):
    # LLMに質問を投げるノード
    ____ = state["user_question"]
    print(f"llm_responder_node: Asking LLM: '{question}'")
    # LLMに渡すメッセージは、過去の履歴全体でも、最新の質問だけでも良い
    # ここでは簡単のため、最新の質問のみをHumanMessageとして渡す
    ____ = ____.invoke([HumanMessage(content=question)]) 
    ____ = response.content
    print(f"llm_responder_node: LLM raw response: '{response_content}'")
    return {____: [response], ____: response_content}

def ____(state: ExtractionState):
    # LLMの応答から情報を抽出するノード
    ____ = state["llm_response_text"]
    # 例: LLMが「日本の首都は東京です。」と答えたら「東京」を抽出
    # ここでは簡単な正規表現で「XXはYYです」のYY部分を抽出試行
    ____ = None
    ____ = re.search(r"(?:は|is)\s*([^。.]+)[.。]?", raw_response) 
    if match:
        ____ = match.group(1).strip()
    
    print(f"data_extractor_node: Raw response: '{raw_response}', Extracted: '{extracted}'")
    return {____: extracted, ____: [AIMessage(content=f"Extracted: {extracted}")]}

# --- グラフ構築 (Graph) ---
____ = ____(ExtractionState)

____.____("capture_question", get_user_question)
____.____("ask_llm", llm_responder_node)
____.____("extract_data", data_extractor_node)

____.____("capture_question")

____.____("capture_question", "ask_llm")
____.____("ask_llm", "extract_data")
____.____("extract_data", END)

____ = ____.compile()

In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

In [ ]:
# --- グラフの実行と結果表示 ---
questions = [
    "日本の首都は何ですか？",
    "フランスの有名な画家の名前を一人教えてください。",
    "今日の天気は？" # これは抽出パターンにマッチしない可能性
]

for q_text in questions:
    print(f"\n--- LLM連携と情報抽出テスト (質問: {q_text}) ---")
    inputs = {"messages": [HumanMessage(content=q_text)]}
    # final_state = None # This was problematic, invoke directly

    # if final_state: # This block was never reached
    #     print(f"最終的な応答: {final_state['messages'][-1].content}")
    #     print(f"  User Question: {final_state.get('user_question')}")
    #     print(f"  LLM Response: {final_state.get('llm_response_text')}")
    #     print(f"  Extracted Info: {final_state.get('extracted_info')}")
    # else:
    final_state_invoked = graph.invoke(inputs, {"recursion_limit": 5})
    print(f"最終的な応答: {final_state_invoked['messages'][-1].content}")
    print(f"  User Question: {final_state_invoked.get('user_question')}")
    print(f"  LLM Response: {final_state_invoked.get('llm_response_text')}")
    print(f"  Extracted Info: {final_state_invoked.get('extracted_info')}")

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")


<details><summary>解答008</summary>

``````python
# 解答008
from typing import TypedDict, Annotated
import re # 正規表現モジュールをインポート
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class ExtractionState(TypedDict):
    messages: Annotated[list, add_messages]
    user_question: str # ユーザーの元の質問
    llm_response_text: str # LLMの生の応答テキスト
    extracted_info: str | None # LLMの応答から抽出された情報

# --- ノード定義 (Nodes) ---
def get_user_question(state: ExtractionState):
    # ユーザーの質問を状態に保存
    last_message_content = state["messages"][-1].content
    print(f"get_user_question: Received question: '{last_message_content}'")
    return {"user_question": last_message_content}

def llm_responder_node(state: ExtractionState):
    # LLMに質問を投げるノード
    question = state["user_question"]
    print(f"llm_responder_node: Asking LLM: '{question}'")
    response = llm.invoke([HumanMessage(content=question)])
    response_content = response.content
    print(f"llm_responder_node: LLM raw response: '{response_content}'")
    return {"messages": [response], "llm_response_text": response_content}

def data_extractor_node(state: ExtractionState):
    # LLMの応答から情報を抽出するノード
    raw_response = state["llm_response_text"]
    extracted = None
    # 改善された正規表現: 「XXはYYです」「XX is YY」のようなパターンや、単に「YYです」のような応答にも対応試行
    # 質問が「日本の首都は何ですか？」で応答が「東京です。」の場合「東京」を抽出
    # 質問が「日本の首都は？」で応答が「東京」の場合「東京」を抽出
    patterns = [
        r"(?:.+は|.+\s*is)\s*(.+?)(?:です|。|\.|\s*for|$)", #「～はXです」「～ is X」
        r"^([^。.]+?)(?:です|。|\.|\s*for|$)" # 文頭から「Xです」
    ]
    for pattern in patterns:
        match = re.search(pattern, raw_response)
        if match:
            extracted = match.group(1).strip()
            if extracted.lower() == state["user_question"].lower().replace("何ですか","").replace("何","зиру").strip("?？") : # 質問自体が答えになるような場合を除外
                 extracted = None # 例：「天気は？」->「晴れです」はOKだが、「天気は？」->「天気」はNG
                 continue
            break
    
    # もし上記で抽出できなかった場合、LLMの応答が単語やフレーズそのものである可能性を考慮
    if not extracted and len(raw_response.split()) < 5 and not state["user_question"].startswith(raw_response): # 短い応答で、質問の繰り返しでない場合
        extracted = raw_response.strip()

    print(f"data_extractor_node: Raw response: '{raw_response}', Extracted: '{extracted}'")
    return {"extracted_info": extracted, "messages": [AIMessage(content=f"Extracted info: {extracted if extracted else 'N/A'}")]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(ExtractionState)

workflow.add_node("capture_question", get_user_question)
workflow.add_node("ask_llm", llm_responder_node)
workflow.add_node("extract_data", data_extractor_node)

workflow.set_entry_point("capture_question")

workflow.add_edge("capture_question", "ask_llm")
workflow.add_edge("ask_llm", "extract_data")
workflow.add_edge("extract_data", END)

graph = workflow.compile()

# --- グラフの実行と結果表示 ---
questions = [
    "日本の首都は何ですか？",
    "フランスの有名な画家の名前を一人教えてください。", # LLMの回答次第で抽出成功/失敗が変わる
    "1+1は何？", # LLMが「2です」と答えれば抽出できるかも
    "今日の天気は？" # 「晴れです」なら「晴れ」を抽出期待
]

for q_text in questions:
    print(f"\n--- LLM連携と情報抽出テスト (質問: {q_text}) ---")
    inputs = {"messages": [HumanMessage(content=q_text)], "llm_response_text": "", "extracted_info": None} # 初期値を設定
    
    final_state_data = graph.invoke(inputs, {"recursion_limit": 5})

    if final_state_data:
        print(f"最終的な応答: {final_state_data['messages'][-1].content}")
        print(f"  User Question: {final_state_data.get('user_question')}")
        print(f"  LLM Response: {final_state_data.get('llm_response_text')}")
        print(f"  Extracted Info: {final_state_data.get('extracted_info')}")
    else:
        print("Could not retrieve final state.")

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")
``````
</details>

<details><summary>解説008</summary>

#### この問題のポイント

*   **コード解説:**
    *   `ExtractionState`には、ユーザーの質問 (`user_question`)、LLMの生の応答テキスト (`llm_response_text`)、そして抽出された情報 (`extracted_info`) を保持するためのキーが定義されています。
    *   `get_user_question`ノード: ユーザーからの最初のメッセージを `user_question` として状態に保存します。
    *   `llm_responder_node`: 保存された `user_question` を使ってLLMに問い合わせを行い、得られた応答を `messages` (AIMessageとして) と `llm_response_text` (生の文字列として) 状態に保存します。
    *   `data_extractor_node`: `llm_response_text` から情報を抽出します。この例では、簡単な正規表現 `re.search(r"(?:は|is)\s*([^。.]+)[.。]?", raw_response)` を使用して、「AはBです」や「A is B」といった形式の文からBの部分を抽出しようと試みています。抽出結果は `extracted_info` として状態に保存されます。正規表現は完璧ではなく、LLMの応答形式によってはうまく抽出できない場合もありますが、ここではLLMの出力後処理の一例として示しています。
    *   グラフは `capture_question` -> `ask_llm` -> `extract_data` -> `END` というシーケンシャルな流れで定義されています。
    *   実行時には、異なる質問を投げ、LLMの応答とそこから抽出された情報（または抽出できなかった場合は `None` や `N/A`）が最終状態に含まれることを確認します。
*   **発展:**
    *   情報抽出の方法は正規表現に限らず、より高度なNLPライブラリ（例: spaCy）や、LangChainが提供するOutput Parsers、あるいは別のLLMコール（Function Calling/Tool Calling対応モデルならより高精度）を使って行うことも考えられます。
    *   抽出に失敗した場合のフォールバック処理（例: ユーザーに再確認を求める、デフォルト値を設定するなど）をグラフに追加することもできます。
---
</details>

### ■ 問題009: グラフの入力と出力のカスタマイズと明確化

LangGraphのグラフを実行する際、`invoke()` や `stream()` メソッドに渡す初期状態の構造と、グラフ全体の最終的な出力状態の構造を意識することが重要です。`StateGraph` に渡す状態クラス（例: `TypedDict`）の定義が、実質的にグラフの入力と出力のスキーマ（型定義）となります。この問題では、入力として複数の情報を受け取り、それらを処理して特定の構造で出力するグラフを作成し、入出力の対応関係を明確に意識します。

*   **学習内容:** `TypedDict` を使ってグラフの状態スキーマを定義する際、どのキーがグラフへの主要な「入力」として期待され、どのキーがグラフからの主要な「出力」として扱われるのかを明確に意識することの重要性を学びます。また、`Optional`型やデフォルト値の扱い方についても触れます。


In [ ]:
# 解答欄009
from ____ import ____, ____, ____, ____
from ____.____ import ____, ____
from ____.____.____ import ____
from ____.____.____ import ____, ____

# --- 状態定義 (入力と出力のスキーマを兼ねる) ---
class ____(____):
    ____: str
    ____: str
    ____: bool

class ____(____):
    # 入力として期待されるキー
    ____: str
    ____: List[str]
    ____: Optional[int]

    # 処理中に使われるキー
    ____: Annotated[list, add_messages]
    ____: int

    # 出力として期待される主要なキー
    ____: Optional[ProcessedData] 
    ____: Optional[str] 

# --- ノード定義 (Nodes) ---
def ____(state: ComplexIOState):
    print(f"initialize_processing: Received item '{state['raw_item_name']}' with details {state['raw_item_details']}")
    return { 
        "messages": [AIMessage(content=f"Initializing processing for {state['raw_item_name']}")],
        "internal_counter": 0,
        "processed_data": None, 
        "error_message": None   
    }

def ____(state: ComplexIOState):
    name = state[____]
    details_count = len(state[____])
    counter = state[____] + 1
    threshold = state.get(____, 0)
    
    log_msg = f"Processing '{name}', detail count: {details_count}, attempt: {counter}, threshold: {threshold}"
    print(f"main_processor: {log_msg}")

    if details_count == 0:
        err_msg = "No details provided."
        return {
            "messages": [AIMessage(content=f"Error: {err_msg}")],
            "error_message": err_msg,
            "internal_counter": counter
        }
    
    if counter > threshold: 
        processed_item = ProcessedData(
            item_id=f"PROC_{name.upper()}_{counter}",
            description=f"Successfully processed {name} with {details_count} details after {counter} attempts.",
            is_processed=True
        )
        return {
            "messages": [AIMessage(content=f"Successfully processed {name}")],
            "processed_data": processed_item,
            "internal_counter": counter
        }
    else:
        return {
            "messages": [AIMessage(content=f"Attempt {counter} for {name} did not meet threshold.")],
            "internal_counter": counter
        }

# --- 条件付きエッジのルーター関数 ---
def ____(state: ComplexIOState):
    if state.get(____):
        print("check_status: Error detected, routing to END.")
        return "__end__" 
    if state.get(____) and state[____][____]:
        print("check_status: Successfully processed, routing to END.")
        return "__end__"
    else:
        print("check_status: Not yet processed or error, routing back to main_processor.")
        return "retry_processing"

# --- グラフ構築 (Graph) ---
____ = ____(ComplexIOState)

____.____("initializer", initialize_processing)
____.____("processor", main_processor)

____.____("initializer")
____.____("initializer", "processor")

____.____(
    "processor",
    check_status,
    {
        "__end__": END,
        "retry_processing": "processor"
    }
)

____ = ____.compile()

In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

In [ ]:
# --- グラフの実行と結果表示 ---
inputs_success = {
    "raw_item_name": "TestItem1",
    "raw_item_details": ["detail A", "detail B"],
    "processing_threshold": 2 
}

inputs_fail_no_details = {
    "raw_item_name": "TestItem2",
    "raw_item_details": [], 
    "processing_threshold": 1
}

inputs_optional_threshold_not_provided = {
    "raw_item_name": "TestItem3",
    "raw_item_details": ["detail C"]
    # processing_threshold is not provided (Optional)
}

test_cases = {
    "Success Case": inputs_success, 
    "Failure Case (No Details)": inputs_fail_no_details,
    "Success Case (Threshold Not Provided)": inputs_optional_threshold_not_provided
}

for case_name, inputs_data in test_cases.items():
    print(f"\n--- I/Oカスタマイズテスト: {case_name} ---")
    current_inputs = inputs_data.copy()
    current_inputs.setdefault("messages", []) 

    final_output_state = graph.invoke(current_inputs, {"recursion_limit": 10})
    print(f"最終的な応答: {final_output_state['messages'][-1].content}")

    if final_output_state.get("processed_data"):
        print(f"  Processed Item ID: {final_output_state['processed_data']['item_id']}")
        print(f"  Processed: {final_output_state['processed_data']['is_processed']}")
    if final_output_state.get("error_message"):
        print(f"  Error: {final_output_state['error_message']}")

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")


<details><summary>解答009</summary>

``````python
# 解答009
from typing import TypedDict, Annotated, List, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (入力と出力のスキーマを兼ねる) ---
class ProcessedData(TypedDict):
    item_id: str
    description: str
    is_processed: bool

class ComplexIOState(TypedDict):
    # 入力として期待されるキー
    raw_item_name: str
    raw_item_details: List[str]
    processing_threshold: Optional[int]

    # 処理中に使われるキー
    messages: Annotated[list, add_messages] # 処理ログ用
    internal_counter: int

    # 出力として期待される主要なキー
    processed_data: Optional[ProcessedData] # 処理結果
    error_message: Optional[str] # エラー発生時のメッセージ

# --- ノード定義 (Nodes) ---
def initialize_processing(state: ComplexIOState):
    print(f"initialize_processing: Received item '{state['raw_item_name']}' with details {state['raw_item_details']}")
    return {
        "messages": [AIMessage(content=f"Initializing processing for {state['raw_item_name']}")],
        "internal_counter": 0,
        "processed_data": None, 
        "error_message": None   
    }

def main_processor(state: ComplexIOState):
    name = state["raw_item_name"]
    details_count = len(state["raw_item_details"])
    counter = state["internal_counter"] + 1
    threshold = state.get("processing_threshold") if state.get("processing_threshold") is not None else 0
    
    log_msg = f"Processing '{name}', detail count: {details_count}, attempt: {counter}, threshold: {threshold}"
    print(f"main_processor: {log_msg}")

    if details_count == 0:
        err_msg = "No details provided."
        return {
            "messages": [AIMessage(content=f"Error: {err_msg}")],
            "error_message": err_msg,
            "internal_counter": counter
        }
    
    if counter > threshold:
        processed_item = ProcessedData(
            item_id=f"PROC_{name.upper()}_{counter}",
            description=f"Successfully processed {name} with {details_count} details after {counter} attempts.",
            is_processed=True
        )
        return {
            "messages": [AIMessage(content=f"Successfully processed {name}")],
            "processed_data": processed_item,
            "internal_counter": counter
        }
    else:
        return {
            "messages": [AIMessage(content=f"Attempt {counter} for {name} did not meet threshold.")],
            "internal_counter": counter
        }

# --- 条件付きエッジのルーター関数 ---
def check_status(state: ComplexIOState):
    if state.get("error_message"):
        print("check_status: Error detected, routing to END.")
        return "__end__" 
    if state.get("processed_data") and state["processed_data"]["is_processed"]:
        print("check_status: Successfully processed, routing to END.")
        return "__end__"
    else:
        print("check_status: Not yet processed or error, routing back to main_processor.")
        return "retry_processing"

# --- グラフ構築 (Graph) ---
workflow = StateGraph(ComplexIOState)

workflow.add_node("initializer", initialize_processing)
workflow.add_node("processor", main_processor)

workflow.set_entry_point("initializer")
workflow.add_edge("initializer", "processor")

workflow.add_conditional_edges(
    "processor",
    check_status,
    {
        "__end__": END,
        "retry_processing": "processor"
    }
)

graph = workflow.compile()

# --- グラフの実行と結果表示 ---
inputs_success = {
    "raw_item_name": "TestItem1",
    "raw_item_details": ["detail A", "detail B"],
    "processing_threshold": 2 
}

inputs_fail_no_details = {
    "raw_item_name": "TestItem2",
    "raw_item_details": [], 
    "processing_threshold": 1
}

inputs_optional_threshold_not_provided = {
    "raw_item_name": "TestItem3",
    "raw_item_details": ["detail C"],
}

test_cases = {
    "Success Case": inputs_success,
    "Failure Case (No Details)": inputs_fail_no_details,
    "Success Case (Threshold Not Provided)": inputs_optional_threshold_not_provided
}

for case_name, inputs_data in test_cases.items():
    print(f"\n--- I/Oカスタマイズテスト: {case_name} ---")
    current_inputs = inputs_data.copy()
    current_inputs.setdefault("messages", []) 

    final_output_state = graph.invoke(current_inputs, {"recursion_limit": 10})
    print(f"最終的な応答: {final_output_state['messages'][-1].content}")

    if final_output_state.get("processed_data"):
        print(f"  Processed Item ID: {final_output_state['processed_data']['item_id']}")
        print(f"  Processed: {final_output_state['processed_data']['is_processed']}")
    if final_output_state.get("error_message"):
        print(f"  Error: {final_output_state['error_message']}")

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")
``````
</details>

<details><summary>解説009</summary>

#### この問題のポイント

*   **コード解説:**
    *   `ComplexIOState` (状態スキーマ):
        *   **入力想定キー:** `raw_item_name`, `raw_item_details`, `processing_threshold` (これは `Optional` なので、入力時に省略可能)。これらはグラフ実行時に `invoke` や `stream` の `inputs` 引数で渡されることが期待されます。
        *   **処理中キー:** `messages`, `internal_counter`。これらは主にグラフ内部の処理やログのために使われ、通常は入力時に指定しません（`messages` は `add_messages` のために空リストで初期化することがあります）。
        *   **出力想定キー:** `processed_data` (処理成功時の結果), `error_message` (エラー発生時の情報)。これらのキーの値が、グラフ実行後の最終的な成果物となります。
    *   `initialize_processing`ノード: 入力値を受け取り、処理に必要な内部状態（`internal_counter`など）や出力用キー（`processed_data`, `error_message`）を初期化します。
    *   `main_processor`ノード: 主要な処理ロジックを担当します。入力された `raw_item_name` や `raw_item_details`、そして `processing_threshold` (入力されなければデフォルト値を使用) に基づいて処理を行い、成功すれば `processed_data` を、失敗すれば `error_message` を更新します。また、処理試行回数を `internal_counter` で管理します。
    *   `check_status`ルーター関数: `error_message` があれば終了。`processed_data` があれば終了。それ以外（まだ処理が完了していない、またはエラーではないが成功もしていない）場合は `main_processor` に戻って処理を続行（リトライ/ループ）します。
    *   グラフ実行時:
        *   `inputs`辞書には、`ComplexIOState`で定義した入力想定キー（`raw_item_name`など）を指定します。
        *   `invoke()` から返される `final_output_state` は、`ComplexIOState` と同じ構造の辞書です。この中から出力想定キー（`processed_data` や `error_message`）の値を確認することで、グラフの実行結果を得ます。
*   **重要な点:**
    *   状態スキーマ（`TypedDict`）は、グラフのインターフェース（入力と出力の形式）を定義する上で中心的な役割を果たします。
    *   入力時にどのキーが必要で、どのキーがオプショナルか、そしてグラフ実行後にどのキーに出力結果が格納されるのかを明確に設計することが、再利用可能で理解しやすいグラフを作る上で重要です。
---
</details>

### ■ 問題010: 複数のLLM呼び出しを含むグラフ

一つのグラフ内で、異なる役割やプロンプトを持つ複数のLLM呼び出しノードを組み込む方法を学びましょう。例えば、最初のLLMがアイデアを生成し、次のLLMがそのアイデアを評価・洗練する、といった連携が考えられます。この問題では、簡単な役割分担を持つ2つのLLMノードを直列に接続します。

*   **学習内容:** 一つのグラフ内に、それぞれ異なるプロンプトや役割を持つ複数のLLM呼び出しノードを配置し、それらを連携させる方法を学びます。これにより、より複雑で多段階の思考や処理を行うエージェントやパイプラインを構築できます。


In [ ]:
# 解答欄010
from ____ import ____, ____, ____
from ____.____ import ____, ____
from ____.____.____ import ____
from ____.____.____ import ____, ____
from ____.____.____ import ____

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class ____(____):
    ____: ____[____, ____]
    ____: str 
    ____: str | ____ 
    ____: str | ____ 

# --- ノード定義 (Nodes) ---
def ____(state: MultiLLMState):
    ____ = state["messages"][-1].content
    print(f"get_topic: Original topic is '{topic}'")
    return {____: topic, ____: [AIMessage(content=f"Topic received: {topic}")]}

def ____(state: MultiLLMState):
    ____ = state["original_topic"]
    print(f"idea_generation_node: Generating idea for topic: '{topic}'")
    
    ____ = ChatPromptTemplate.from_messages([
        ("system", "あなたは新しいアイデアを生み出すのが得意なAIです。与えられたトピックに関して、ユニークで面白いアイデアを一つ提案してください。アイデアは簡潔に一行で述べてください。"),
        ("human", "トピック: {topic}")
    ])
    
    ____ = prompt_template_idea | llm 
    ____ = chain.invoke({"topic": topic})
    ____ = response.content.strip()
    
    print(f"idea_generation_node: Generated idea: '{idea}'")
    return {____: idea, ____: [AIMessage(content=f"Generated Idea: {idea}")]}

def ____(state: MultiLLMState):
    ____ = state["generated_idea"]
    if not idea:
        return {____: [AIMessage(content="No idea to evaluate.")], ____: "N/A"}
        
    print(f"idea_evaluation_node: Evaluating idea: '{idea}'")
    
    ____ = ChatPromptTemplate.from_messages([
        ("system", "あなたはアイデアを客観的に評価するのが得意なAIです。与えられたアイデアについて、その実現可能性と面白さを評価し、短いコメントを述べてください。"),
        ("human", "評価対象のアイデア: {idea}")
    ])
    
    ____ = prompt_template_eval | llm 
    ____ = chain.invoke({"idea": idea})
    ____ = response.content.strip()
    
    print(f"idea_evaluation_node: Evaluation: '{evaluation}'")
    return {____: evaluation, ____: [AIMessage(content=f"Evaluation: {evaluation}")]}

# --- グラフ構築 (Graph) ---
____ = ____(MultiLLMState)

____.____("capture_topic", get_topic)
____.____("generate_idea", idea_generation_node)
____.____("evaluate_idea", idea_evaluation_node)

____.____("capture_topic")

____.____("capture_topic", "generate_idea")
____.____("generate_idea", "evaluate_idea")
____.____("evaluate_idea", END)

____ = ____.compile()

In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

In [ ]:
# --- グラフの実行と結果表示 ---
topics_to_test = [
    "新しい料理のレシピ",
    "未来の交通手段",
    "週末の過ごし方"
]

for topic_text in topics_to_test:
    print(f"\n--- 複数LLM連携テスト (トピック: {topic_text}) ---")
    inputs = {
        "messages": [HumanMessage(content=topic_text)],
        "original_topic": "", 
        "generated_idea": None,
        "evaluated_idea": None
    }
    final_state = graph.invoke(inputs, {"recursion_limit": 5})
    print(f"最終的な応答: {final_state['messages'][-1].content}")
    print(f"  Original Topic: {final_state.get('original_topic')}")
    print(f"  Generated Idea: {final_state.get('generated_idea')}")
    print(f"  Evaluated Idea: {final_state.get('evaluated_idea')}")

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")


<details><summary>解答010</summary>

``````python
# 解答010
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import Image, display

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
class MultiLLMState(TypedDict):
    messages: Annotated[list, add_messages]
    original_topic: str # ユーザーからの最初のトピック
    generated_idea: str | None # アイデア生成LLMの出力
    evaluated_idea: str | None # アイデア評価LLMの出力

# --- ノード定義 (Nodes) ---
def get_topic(state: MultiLLMState):
    topic = state["messages"][-1].content
    print(f"get_topic: Original topic is '{topic}'")
    return {"original_topic": topic, "messages": [AIMessage(content=f"Topic received: {topic}")]}

def idea_generation_node(state: MultiLLMState):
    topic = state["original_topic"]
    print(f"idea_generation_node: Generating idea for topic: '{topic}'")
    
    prompt_template_idea = ChatPromptTemplate.from_messages([
        ("system", "あなたは新しいアイデアを生み出すのが得意なAIです。与えられたトピックに関して、ユニークで面白いアイデアを一つ提案してください。アイデアは簡潔に一行で述べてください。"),
        ("human", "トピック: {topic}")
    ])
    
    chain = prompt_template_idea | llm 
    response = chain.invoke({"topic": topic})
    idea = response.content.strip()
    
    print(f"idea_generation_node: Generated idea: '{idea}'")
    return {"generated_idea": idea, "messages": [AIMessage(content=f"Generated Idea: {idea}")]}

def idea_evaluation_node(state: MultiLLMState):
    idea = state["generated_idea"]
    if not idea:
        return {"messages": [AIMessage(content="No idea to evaluate.")], "evaluated_idea": "N/A"}
        
    print(f"idea_evaluation_node: Evaluating idea: '{idea}'")
    
    prompt_template_eval = ChatPromptTemplate.from_messages([
        ("system", "あなたはアイデアを客観的に評価するのが得意なAIです。与えられたアイデアについて、その実現可能性と面白さを評価し、短いコメントを述べてください。"),
        ("human", "評価対象のアイデア: {idea}")
    ])
    
    chain = prompt_template_eval | llm 
    response = chain.invoke({"idea": idea})
    evaluation = response.content.strip()
    
    print(f"idea_evaluation_node: Evaluation: '{evaluation}'")
    return {"evaluated_idea": evaluation, "messages": [AIMessage(content=f"Evaluation: {evaluation}")]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(MultiLLMState)

workflow.add_node("capture_topic", get_topic)
workflow.add_node("generate_idea", idea_generation_node)
workflow.add_node("evaluate_idea", idea_evaluation_node)

workflow.set_entry_point("capture_topic")

workflow.add_edge("capture_topic", "generate_idea")
workflow.add_edge("generate_idea", "evaluate_idea")
workflow.add_edge("evaluate_idea", END)

graph = workflow.compile()

# --- グラフの実行と結果表示 ---
topics_to_test = [
    "新しい料理のレシピ",
    "未来の交通手段",
    "週末の過ごし方"
]

for topic_text in topics_to_test:
    print(f"\n--- 複数LLM連携テスト (トピック: {topic_text}) ---")
    inputs = {
        "messages": [HumanMessage(content=topic_text)],
        "original_topic": "", 
        "generated_idea": None,
        "evaluated_idea": None
    }
    final_state = graph.invoke(inputs, {"recursion_limit": 5})
    print(f"最終的な応答: {final_state['messages'][-1].content}")
    print(f"  Original Topic: {final_state.get('original_topic')}")
    print(f"  Generated Idea: {final_state.get('generated_idea')}")
    print(f"  Evaluated Idea: {final_state.get('evaluated_idea')}")

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"Graph visualization failed: {e}")
``````
</details>

<details><summary>解説010</summary>

#### この問題のポイント

*   **コード解説:**
    *   `MultiLLMState`には、ユーザーからの最初のトピック (`original_topic`)、最初のLLMが生成したアイデア (`generated_idea`)、そして二番目のLLMが評価した結果 (`evaluated_idea`) を保持するキーが定義されています。
    *   `get_topic`ノード: ユーザーの入力を `original_topic` として状態に保存します。
    *   `idea_generation_node`: `original_topic` に基づいて、アイデア生成用のプロンプト (`prompt_template_idea`) を使用してLLMを呼び出し、結果を `generated_idea` に保存します。
    *   `idea_evaluation_node`: `generated_idea` に基づいて、アイデア評価用のプロンプト (`prompt_template_eval`) を使用してLLMを呼び出し（ここでも同じ `llm` インスタンスを使用していますが、プロンプトが異なるため役割が変わります）、結果を `evaluated_idea` に保存します。
    *   グラフは `capture_topic` -> `generate_idea` -> `evaluate_idea` -> `END` という直列な流れで、各ステップで状態が更新されていきます。
    *   各LLM呼び出しノード内では、`langchain_core.prompts.ChatPromptTemplate` を使ってそのノード専用のプロンプトを定義し、共通の `llm` インスタンスと組み合わせて (例: `chain = prompt_template | llm`) LLM呼び出しを行っています。これにより、同じLLMモデルでも異なる指示を与えることで、多様な処理を実現できます。
*   **応用例:**
    *   リサーチアシスタント: 質問受け付け -> 情報検索プロンプトでLLM -> 要約プロンプトでLLM -> 報告書作成プロンプトでLLM。
    *   コード生成・レビュー: 要件定義 -> コード生成LLM -> 生成コード評価LLM -> 修正指示LLM。
    *   このように、タスクを細分化し、各サブタスクに特化したプロンプトを持つLLMノードを連携させることで、より高品質な結果を得ることが期待できます。
---
</details>

### ■ 問題011: 第1章のまとめ - 簡単なQ&Aボットの改善

第1章で学んだ様々な要素（状態管理、LLM連携、条件分岐、ループ、情報抽出など）を組み合わせて、問題004で作成したシンプルなチャットボットを改善してみましょう。このQ&Aボットは、ユーザーの質問のタイプ（例: 単純な挨拶、知識を問う質問、不明な質問）を判別し、応答を変化させたり、会話の回数をカウントしたりする機能を持つようにします。

*   **学習内容:** 第1章で学んだ複数の概念（`StateGraph`の定義、`TypedDict`による状態管理、ノードとエッジの追加、LLM呼び出し、条件付きエッジによる分岐、ループ（会話ターン数制限による間接的なループ制御）、状態キーの更新）を統合し、少し複雑な対話型のQ&Aボットを構築します。これにより、LangGraphの基本的な要素を組み合わせて実用的なアプリケーションを作成する流れを体験します。


In [ ]:
# 解答欄011
from ____ import ____, ____, ____, ____
import ____
from ____.____ import ____, ____
from ____.____.____ import ____
from ____.____.____ import ____, ____
from ____.____.____ import ____

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
____ = Literal["greeting", "knowledge_q", "opinion_q", "unknown_q"]

class ____(____):
    ____: Annotated[list, add_messages]
    ____: str 
    ____: Optional[QuestionCategory] 
    ____: Optional[str] 
    ____: int 
    ____: int 
    ____: Optional[str]

# --- ノード定義 (Nodes) ---
def ____(state: AdvancedQABotState):
    user_message = state["messages"][-1].content
    current_turns = state.get(____, 0) + 1
    print(f"capture_input_and_increment_turn: User: '{user_message}', Turn: {current_turns}")
    return { 
        "user_input": user_message,
        "conversation_turns": current_turns,
        "question_category": None, 
        "llm_response": None, 
        "error_message": None 
    }

def ____(state: AdvancedQABotState):
    text = state[____].lower()
    category: QuestionCategory = "unknown_q"
    
    if any(greet in text for greet in ["こんにちは", "やあ", "どうも", "hello", "hi"]):
        category = "greeting"
    elif any(q_word in text for q_word in ["とは", "何ですか", "教えて", "なぜ", "what is", "tell me about", "why"]) or "?" in text:
        if any(opinion_word in text for opinion_word in ["どう思う", "あなたの意見は", "what do you think about"]):
             category = "opinion_q"
        else:
            category = "knowledge_q"
    
    print(f"categorize_question_node: Input '{text}' categorized as '{category}'")
    return {____: category, ____: [AIMessage(content=f"Category determination: {category}")]}

def ____(state: AdvancedQABotState):
    response = "こんにちは！何かお手伝いできることはありますか？"
    print(f"greeting_responder_node: Responding with '{response}'")
    return {____: response}

def ____(state: AdvancedQABotState):
    question = state["user_input"]
    print(f"knowledge_llm_node: Asking LLM (knowledge): '{question}'")
    response_obj = ____.invoke([HumanMessage(content=question)]) 
    response_text = response_obj.content.strip()
    print(f"  LLM response: {response_text}")
    return {____: response_text}

def ____(state: AdvancedQABotState):
    question = state["user_input"]
    print(f"opinion_llm_node: Asking LLM (opinion): '{question}'")
    prompt = ____.from_messages([
        ("system", "あなたは様々なトピックについて個人的な意見を述べることができるAIです。客観的な事実とあなたの意見を区別して話してください。"),
        ("human", "{user_question}")
    ])
    chain = prompt | ____
    response_obj = chain.invoke({"user_question": question})
    response_text = response_obj.content.strip()
    print(f"  LLM response: {response_text}")
    return {____: response_text}

def ____(state: AdvancedQABotState):
    response = "申し訳ありませんが、ご質問の意図を正確に理解できませんでした。もう少し具体的に、または別の言葉で質問していただけますでしょうか？"
    print(f"unknown_question_node: Responding with '{response}'")
    return {____: response}

def ____(state: AdvancedQABotState):
    final_resp = state.get(____, "(AI did not generate a response for some reason)")
    print(f"final_response_node: Final AI response to be added to messages: '{final_resp}'")
    return {____: [AIMessage(content=final_resp)]}

# --- ルーター関数 ---
def ____(state: AdvancedQABotState):
    category = state.get("question_category")
    print(f"route_by_category: Routing based on category '{category}'")
    if category == "greeting": return "greeting_responder"
    if category == "knowledge_q": return "knowledge_llm"
    if category == "opinion_q": return "opinion_llm"
    return "unknown_responder"

def ____(state: AdvancedQABotState):
    current_turns = state.get("conversation_turns", 0)
    max_t = state.get("max_turns", 3) 
    if current_turns >= max_t:
        print(f"check_conversation_limit: Max turns ({max_t}) reached for this interaction. Ending conversation.")
        return "__end__" 
    print(f"check_conversation_limit: Turn {current_turns}/{max_t}. Continuing to categorize.")
    return "continue_to_categorizer"

# --- グラフ構築 (Graph) ---
____ = ____(AdvancedQABotState)

____.____("input_handler", capture_input_and_increment_turn)
____.____("categorizer", categorize_question_node)
____.____("greeting_responder", greeting_responder_node)
____.____("knowledge_llm", knowledge_llm_node)
____.____("opinion_llm", opinion_llm_node)
____.____("unknown_responder", unknown_question_node)
____.____("final_responder", final_response_node)

____.____("input_handler")

____.____(
    "input_handler",
    check_conversation_limit,
    {
        "continue_to_categorizer": "categorizer",
        "__end__": END 
    }
)

____.____(
    "categorizer",
    route_by_category,
    {
        "greeting_responder": "greeting_responder",
        "knowledge_llm": "knowledge_llm",
        "opinion_llm": "opinion_llm",
        "unknown_responder": "unknown_responder"
    }
)

____.____("greeting_responder", "final_responder")
____.____("knowledge_llm", "final_responder")
____.____("opinion_llm", "final_responder")
____.____("unknown_responder", "final_responder")
____.____("final_responder", END)

____ = ____.compile()

In [ ]:
# --- グラフの可視化 ---
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

In [ ]:
# --- グラフの実行と結果表示 (インタラクティブテスト) ---
chat_history_for_bot = []
max_dialogue_turns = 3 

print(f"--- 第1章まとめ Q&Aボット (最大{max_dialogue_turns}往復) ---")
print("チャットを開始します。'exit' または 'quit' で終了します。")

for i in range(max_dialogue_turns):
    user_text = input(f"あなた (Turn {i + 1}): ")
    if user_text.lower() in ["exit", "quit"]:
        print("チャットを終了します。")
        break
    
    chat_history_for_bot.append(HumanMessage(content=user_text))
    
    current_invoke_inputs = {
        "messages": chat_history_for_bot,
        "max_turns": max_dialogue_turns,
    }
    if i == 0: 
        current_invoke_inputs["conversation_turns"] = 0
    # For subsequent turns, conversation_turns will be carried over from the previous state
    # or re-initialized by capture_input_and_increment_turn if not present.

    try:
        final_bot_state = graph.invoke(current_invoke_inputs, {"recursion_limit": 25})

        if final_bot_state and final_bot_state.get("messages"):
            ai_messages = [m for m in final_bot_state["messages"] if isinstance(m, AIMessage)]
            # The actual response to the user should be the last one added by final_responder
            # which is the last AIMessage in the history if the graph completed that far.
            if ai_messages and final_bot_state.get("llm_response") is not None: # Check if llm_response was set
                bot_actual_response = final_bot_state.get("llm_response")
                print(f"AIボット: {bot_actual_response}")
                chat_history_for_bot.append(AIMessage(content=bot_actual_response))
            elif not ai_messages and final_bot_state.get("conversation_turns",0) >= final_bot_state.get("max_turns", max_dialogue_turns):
                 print("AIボット: (最大会話ターン数に達しましたので終了します。)") # Explicit message for max turns reached at END
                 break
            else:
                print("AIボット: (応答がありませんでした。)")
                # If the graph ended due to max_turns before final_responder, there might not be a new AIMessage.
                # Check if it's due to max_turns.
                if final_bot_state.get("conversation_turns",0) >= final_bot_state.get("max_turns", max_dialogue_turns):
                    break # End chat if max turns hit and graph ended
        else:
            print("AIボット: (状態取得エラーまたは会話終了)")
            break

        if final_bot_state.get("conversation_turns", 0) >= final_bot_state.get("max_turns", max_dialogue_turns) and not final_bot_state.get("llm_response") : # type: ignore
             # This case handles if check_conversation_limit routes directly to END
             print("(最大会話ターン数に達しましたので終了します。)")
             break
            
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        import traceback
        traceback.print_exc()
        break
else: 
    print("\n最大会話往復数に達したのでチャットを終了します。")

print("\n--- 最終的なチャット履歴 (ボットとの対話) ---")
for msg in chat_history_for_bot:
    print(f"  {msg.type.upper()}: {msg.content}")

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}")


<details><summary>解答011</summary>

``````python
# 解答011
from typing import TypedDict, Annotated, Literal, Optional
import re
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import Image, display

# ノートブック冒頭で`llm`変数が初期化されている前提

# --- 状態定義 (State) ---
QuestionCategory = Literal["greeting", "knowledge_q", "opinion_q", "unknown_q"]

class AdvancedQABotState(TypedDict):
    messages: Annotated[list, add_messages]
    user_input: str 
    question_category: Optional[QuestionCategory] 
    llm_response: Optional[str] 
    conversation_turns: int 
    max_turns: int 
    error_message: Optional[str]

# --- ノード定義 (Nodes) ---
def capture_input_and_increment_turn(state: AdvancedQABotState):
    user_message = state["messages"][-1].content
    # conversation_turnsはinvokeの入力で初期値を渡すか、.getで安全に取得
    current_turns = state.get("conversation_turns", 0) + 1 
    print(f"capture_input_and_increment_turn: User: '{user_message}', Turn: {current_turns}")
    return {
        "user_input": user_message,
        "conversation_turns": current_turns,
        "question_category": None, 
        "llm_response": None, 
        "error_message": None 
    }

def categorize_question_node(state: AdvancedQABotState):
    text = state["user_input"].lower()
    category: QuestionCategory = "unknown_q"
    
    if any(greet in text for greet in ["こんにちは", "やあ", "どうも", "hello", "hi"]):
        category = "greeting"
    elif any(q_word in text for q_word in ["とは", "何ですか", "教えて", "なぜ", "what is", "tell me about", "why"]) or "?" in text:
        if any(opinion_word in text for opinion_word in ["どう思う", "あなたの意見は", "what do you think about"]):
             category = "opinion_q"
        else:
            category = "knowledge_q"
    
    print(f"categorize_question_node: Input '{text}' categorized as '{category}'")
    return {"question_category": category, "messages": [AIMessage(content=f"Category determination: {category}")]}

def greeting_responder_node(state: AdvancedQABotState):
    response = "こんにちは！何かお手伝いできることはありますか？"
    print(f"greeting_responder_node: Responding with '{response}'")
    return {"llm_response": response}

def knowledge_llm_node(state: AdvancedQABotState):
    question = state["user_input"]
    print(f"knowledge_llm_node: Asking LLM (knowledge): '{question}'")
    response_obj = llm.invoke([HumanMessage(content=question)])
    response_text = response_obj.content.strip()
    print(f"  LLM response: {response_text}")
    return {"llm_response": response_text}

def opinion_llm_node(state: AdvancedQABotState):
    question = state["user_input"]
    print(f"opinion_llm_node: Asking LLM (opinion): '{question}'")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "あなたは様々なトピックについて個人的な意見を述べることができるAIです。客観的な事実とあなたの意見を区別して話してください。"),
        ("human", "{user_question}")
    ])
    chain = prompt | llm
    response_obj = chain.invoke({"user_question": question})
    response_text = response_obj.content.strip()
    print(f"  LLM response: {response_text}")
    return {"llm_response": response_text}

def unknown_question_node(state: AdvancedQABotState):
    response = "申し訳ありませんが、ご質問の意図を正確に理解できませんでした。もう少し具体的に、または別の言葉で質問していただけますでしょうか？"
    print(f"unknown_question_node: Responding with '{response}'")
    return {"llm_response": response}

def final_response_node(state: AdvancedQABotState):
    final_resp = state.get("llm_response", "(AI did not generate a response for some reason)")
    print(f"final_response_node: Final AI response to be added to messages: '{final_resp}'")
    return {"messages": [AIMessage(content=final_resp)]}

def route_by_category(state: AdvancedQABotState):
    category = state.get("question_category")
    print(f"route_by_category: Routing based on category '{category}'")
    if category == "greeting": return "greeting_responder"
    if category == "knowledge_q": return "knowledge_llm"
    if category == "opinion_q": return "opinion_llm"
    return "unknown_responder"

def check_conversation_limit(state: AdvancedQABotState):
    current_turns = state.get("conversation_turns", 0)
    max_t = state.get("max_turns", 3)
    if current_turns >= max_t:
        print(f"check_conversation_limit: Max turns ({max_t}) reached. Routing to END.")
        return "__end__" 
    print(f"check_conversation_limit: Turn {current_turns}/{max_t}. Continuing to categorize.")
    return "continue_to_categorizer"

workflow = StateGraph(AdvancedQABotState)
workflow.add_node("input_handler", capture_input_and_increment_turn)
workflow.add_node("categorizer", categorize_question_node)
workflow.add_node("greeting_responder", greeting_responder_node)
workflow.add_node("knowledge_llm", knowledge_llm_node)
workflow.add_node("opinion_llm", opinion_llm_node)
workflow.add_node("unknown_responder", unknown_question_node)
workflow.add_node("final_responder", final_response_node)
workflow.set_entry_point("input_handler")
workflow.add_conditional_edges(
    "input_handler",
    check_conversation_limit,
    {
        "continue_to_categorizer": "categorizer",
        "__end__": END 
    }
)
workflow.add_conditional_edges(
    "categorizer",
    route_by_category,
    {
        "greeting_responder": "greeting_responder",
        "knowledge_llm": "knowledge_llm",
        "opinion_llm": "opinion_llm",
        "unknown_responder": "unknown_responder"
    }
)
workflow.add_edge("greeting_responder", "final_responder")
workflow.add_edge("knowledge_llm", "final_responder")
workflow.add_edge("opinion_llm", "final_responder")
workflow.add_edge("unknown_responder", "final_responder")
workflow.add_edge("final_responder", END)
graph = workflow.compile()

chat_history_for_bot = []
max_dialogue_turns = 3
print(f"--- 第1章まとめ Q&Aボット (最大{max_dialogue_turns}往復) ---")
print("チャットを開始します。'exit' または 'quit' で終了します。")
for i in range(max_dialogue_turns):
    user_text = input(f"あなた (Turn {i + 1}): ")
    if user_text.lower() in ["exit", "quit"]:
        print("チャットを終了します。")
        break
    
    chat_history_for_bot.append(HumanMessage(content=user_text))
    current_invoke_inputs = {
        "messages": chat_history_for_bot,
        "max_turns": max_dialogue_turns,
        "conversation_turns": i # Pass current turn index for the graph's logic
    }
    try:
        final_bot_state = graph.invoke(current_invoke_inputs, {"recursion_limit": 25})
        if final_bot_state and final_bot_state.get("messages"):
            # Check if the graph execution led to a response being set in llm_response
            if final_bot_state.get("llm_response") is not None:
                bot_actual_response = final_bot_state["llm_response"]
                print(f"AIボット: {bot_actual_response}")
                chat_history_for_bot.append(AIMessage(content=bot_actual_response))
            # If max_turns was hit and routed to END, llm_response might not be set by final_responder
            elif final_bot_state.get("conversation_turns", 0) >= final_bot_state.get("max_turns", max_dialogue_turns):
                 print("AIボット: (最大会話ターン数に達しました。)")
                 break 
            else:
                print("AIボット: (予期せぬ状態で応答がありませんでした。)")
                break
        else:
            print("AIボット: (会話が終了しました。)")
            break
        # Check again if max turns was reached by the graph's internal logic
        if final_bot_state.get("conversation_turns", 0) >= final_bot_state.get("max_turns", max_dialogue_turns):
             break # Exit the loop if graph decided it's max_turns
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        import traceback
        traceback.print_exc()
        break
else:
    print("\n最大会話往復数に達したのでチャットを終了します。")
print("\n--- 最終的なチャット履歴 (ボットとの対話) ---")
for msg in chat_history_for_bot:
    print(f"  {msg.type.upper()}: {msg.content}")
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}")
``````
</details>